In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import animation
from IPython.display import HTML
from collections import deque

In [ ]:
original_maze = [
    ['S', 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0],
    [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0],
    [0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0],
    [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
    [0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0],
    [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0],
    [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
    [0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0],
    [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 'G'],
]

In [ ]:
def generate_random_maze(rows, cols, wall_prob=0.3):
    maze = np.random.choice([0, 1], size=(rows, cols), p=[1 - wall_prob, wall_prob])
    maze[0][0] = 2
    maze[rows - 1][cols - 1] = 3 
    return np.array(maze)

numerized_maze_random = generate_random_maze(10, 10)


In [ ]:
def to_numeric_grid(maze):
    mapping = {'S':2, 'G':3, '1':1, '0':0, 1:1, 0:0}
    numeric_grid = np.array([[mapping.get(element,100) for element in row] for row in maze],dtype=int)
    return numeric_grid

numerized_maze = to_numeric_grid(original_maze)

In [ ]:
def find_pos(maze,value):
    for i in range(len(maze)): #iterating over rows in the maze
        for j in range(len(maze[i])): #iterating over elements in a row
            if maze[i][j]==value:
                return (i,j)
    return None

In [ ]:
def visualize_maze(maze):
    start_pos=find_pos(maze,2)
    goal_pos=find_pos(maze,3)

    cmap = colors.ListedColormap(['white', 'black'])
    bounds = [-0.5, 0.5, 1.5]
    norm = colors.BoundaryNorm(bounds, cmap.N)

    plt.figure(figsize=(10,10))
    plt.imshow(maze, cmap=cmap,norm=norm)

    plt.scatter(start_pos[0],start_pos[1],s=150,c='green',label='Start(S)')
    plt.scatter(goal_pos[0],goal_pos[1],s=150,c='red',label='Goal(G)')
    plt.legend()

    plt.xticks([])
    plt.yticks([])

    plt.show()

visualize_maze(numerized_maze)

In [ ]:
def get_neighbors(maze, r, c):
    neighbors = []
    directions = [(-1,0), (1,0), (0,-1), (0,1)]
    for dr, dc in directions:
        nr, nc = r + dr, c + dc
        if 0 <= nr < maze.shape[0] and 0 <= nc < maze.shape[1] and maze[nr, nc] != 1:
            neighbors.append((nr, nc))
    return neighbors

In [ ]:

def search_maze_animated(maze, start, goal, algorithm='bfs'):
    rows, cols = maze.shape
    new_maze = maze.copy()
    visited = set()
    parent = {}

    if algorithm == 'bfs':
        frontier = deque([(start, 0)])
    elif algorithm == 'dfs':
        frontier = [(start, 0)]
    else:
        raise ValueError("Algorithm must be 'bfs' or 'dfs'")

    frames = [new_maze.copy()]
    path_length = 0
    t0 = time.time()
    while frontier:
        if algorithm == 'bfs':
            (r, c), depth = frontier.popleft()
        else:  # DFS
            (r, c), depth = frontier.pop()

        if (r, c) == goal:
            cur = goal
            while cur != start:
                if new_maze[cur] not in [2, 3]:
                    new_maze[cur] = 5  # Blue path
                    frames.append(new_maze.copy())
                cur = parent[cur]
                path_length += 1
            t1 = time.time()
            exec_time = t1 - t0
            print(f"Visited nodes: {visited}")
            print(f"Total visited nodes: {len(visited)}")
            print(f"Execution time: {exec_time:.5f} seconds")
            return frames, True

        if (r, c) in visited:
            continue
        visited.add((r, c))

        if new_maze[r, c] not in [2, 3]:
            new_maze[r, c] = 4  # Green visited
            frames.append(new_maze.copy())

        for nr, nc in get_neighbors(maze, r, c):
            if (nr, nc) not in visited:
                parent[(nr, nc)] = (r, c)
                if algorithm == 'bfs':
                    frontier.append(((nr, nc), depth + 1))
                else:  # DFS
                    frontier.append(((nr, nc), depth + 1))

    return frames, False


In [ ]:
maze = numerized_maze
# maze_with_sg = numerized_maze
start = (1, 1)
goal = (maze.shape[0]-2, maze.shape[1]-2)

algorithm = input("Enter algorithm (BFS or DFS): ").strip().lower()
frames, found = search_maze_animated(maze, start, goal, algorithm=algorithm)  # or 'dfs'

# Visualization
search_cmap = mcolors.ListedColormap(['white', 'black', 'orange', 'red', 'green', 'blue'])
search_bounds = [0,1,2,3,4,5,6]
search_norm = mcolors.BoundaryNorm(search_bounds, search_cmap.N)

fig, ax = plt.subplots(figsize=(6, 6))
im = ax.imshow(frames[0], cmap=search_cmap, norm=search_norm)
plt.axis('off')
plt.title(f"{algorithm.upper()} Maze Solving Animation")

def animate(i):
    im.set_data(frames[i])
    return [im]

ani = animation.FuncAnimation(fig, animate, frames=len(frames), interval=30, blit=True)
plt.close(fig)
HTML(ani.to_jshtml())
